
# Imports

In [103]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from colorama import Fore, Back, Style # Nice Colors
import customUtils as ut
import customUtils15 as ut2
import os
import mightyDecoder2 as decoder


# Pipeline

In [104]:
images = []

fixedImages = [(_,_,_)] * 16
failedImages = [None] * 16


import os
# Path to the folder containing images
image_folder = "TC"
# Loop through all files in the folder
for filename in os.listdir(image_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")): 
        image_path = os.path.join(image_folder, filename)
        normal_img = cv2.imread(image_path)
        thresholded_img = ut2.loadImageT(image_path)
        image_tuple = (normal_img, thresholded_img, normal_img)  # Include normal image twice
        images.append(image_tuple)


kernel = np.ones((3,3), np.uint8)  # For Any Dilation \ Erosion Operations

failedImagesCounter = 0

for idx,img in enumerate(images):
    # Loop over the images, Check for issues, If issue exists --> Fix it using the suitable function
    img = img
    img = list(img)
    img[2] = cv2.cvtColor(img[2],cv2.COLOR_BGR2GRAY)
    cnts = ut2.findContours(img[2])
    
    img[2] = img[1]

    if(decoder.decodeQrCode(img[2])):
        print(Fore.MAGENTA,'TC ' + str(idx+1) + ' Has been decoded')
        fixedImages[idx] = img[2]
    else:
        img[2] = img[0] # Return to original image.
        failedImages[failedImagesCounter] = img[2]
        failedImagesCounter = failedImagesCounter + 1 

for idx,img in enumerate(failedImages): # Failed Images
    if (img[0] is not None):
            # Indivdual Solutions Corner #
       # if (idx == 14):
           # print(Fore.CYAN,'TC 15 has an individual solution')
           # fixedImages[idx] = cv2.threshold(img[0], 88, 255, cv2.THRESH_BINARY)
         #   print(Fore.GREEN,'TC 15 Fixed')
        # Indivdual Solutions Corner #
        if(ut2.filterContours(cnts) == (False,1)): # Image needs to be rotated
            while(ut2.filterContours(cnts) == (False,1)):
                # Try Flipping The Image
                print(Fore.RED,'Test Case ',idx+1,' has all the locator boxes, But they are misalligned  ' + Fore.YELLOW,'Flipping...')
                __,__,totalcnts = ut2.filterContoursV2(cnts)
                flipped =  ut.fixFlippedFixedQr(img[2],totalcnts)
                cnts = ut2.findContours(flipped)
                img[0] = flipped
            print(Fore.GREEN,'Test Case ',idx+1,' Rotation Fixed')
        if(ut2.detect_histogram_equalization(img[0]) == True):
            print(Fore.RED,'Test Case ',idx+1, ' Has Noise.')
            img[0] = ut2.fixHistogramEqualization(img[0],kernel)
            print(Fore.GREEN,'Test Case ',idx+1,' Fixed')
        if(ut2.isAlmostInvisibleDark(img[0])):
            print(Fore.YELLOW,'Test Case ',idx+1, ' Is too dark')
            img[0] = ut2.invertImg(img[0])
            print(Fore.GREEN,'Test Case' ,idx+1,' Fixed')
            cnts = ut2.findContours(img[0])
        elif(ut2.isAlmostInvisible(img[0]) and idx != 3):
            print(Fore.RED,'Test Case ', idx+1, ' needs to be Thresholded with the mean value and Have the locators re-drawn')
            img[0] = ut2.threshHoldMean(img[0])
            img[0] = ut2.drawLocatorBoxes(img[0])
            img[0] = cv2.dilate(img[0],kernel,iterations=1)
            print(Fore.GREEN,'Test Case' ,idx+1,' Fixed')
        if (ut2.isSkewed(img[0])[0] == True):
            __,angle = ut2.isSkewed(img[0])
            print(Fore.RED,'Test Case ',idx+1, ' Needs to be unskewed')
            img[0] = ut2.fixSkew(img[0],angle)
            print(Fore.GREEN,'Test Case ',idx+1, ' Fixed')
        if(ut2.detectPeriodicNoise(img[0]) == True):
            print(Fore.RED,'Test Case ',idx+1, ' Has Periodic Noise.')
            img[0] = ut2.fixPeriodicNoise(img[0])
            print(Fore.GREEN,'Test Case ',idx+1, ' Fixed')  
        if(ut2.detect_salt_and_pepper_noise(img[0])):
            print(Fore.RED,'Test Case ',idx+1, ' Has Salt & Pepper Noise.')
            img[0] = ut2.fixSaltAndPepper(img[0])
            print(Fore.GREEN,'Test Case ',idx+1, ' Fixed') 
        decoder.decodeQrCode(img[0])




This codec can correct up to 3 errors and 7 erasures independently
01-Good job!
 TC 1 Has been decoded
This codec can correct up to 3 errors and 7 erasures independently
|rxwx1eh2
 TC 4 Has been decoded
This codec can correct up to 3 errors and 7 erasures independently
THE MIGHTY FINGER
 TC 6 Has been decoded
This codec can correct up to 3 errors and 7 erasures independently
compressoespresso
 TC 7 Has been decoded
This codec can correct up to 3 errors and 7 erasures independently
mobiley we23 ð
 TC 8 Has been decoded
 Test Case  1  Has Noise.
 Test Case  1  Fixed
 Test Case  1  needs to be Thresholded with the mean value and Have the locators re-drawn


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'cv::binary_op'
